In [20]:
from utils.llms.gemini import GeminiJsonEngine, GeminiSimpleChatEngine

from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
import os

In [22]:
# For Gemini
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/debasmitroy/Desktop/programming/temp_deployment/key.json"
os.environ["GOOGLE_CLOUD_PROJECT"] = "openserve-0"
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1" 

In [ ]:
class TenorWiseFinSummary(BaseModel):
    """
    This tool is designed to provide tenor wise summary of the financial profit and loss movements
    """
    shortterm_summary: str = Field("No shortterm entries found", title="Summary of p&l and risk movements in the short term")
    midterm_summary: str = Field("No midterm entries found", title="Summary of p&l and risk movements in the mid term")
    longterm_summary: str = Field("No longterm entries found", title="Summary of p&l and risk movements in the long term")

In [54]:
tenor_wise_summary_engnine = GeminiJsonEngine(
                                    model_name="gemini-2.0-flash-001",
                                    basemodel=TenorWiseFinSummary,
                                    temperature=0.5,
                                    max_output_tokens=256,
                                    systemInstructions="""You are financial expert. Please provide a summary of the profit and loss movements in the short term, mid term and long term.
Short term is defined tenor < 2y , mid term is defined as 2y < tenor < 10y and long term is defined as tenor > 10y. Your task to create a extensive summary of the profit and loss movements in each tenors.
User will give a list of p&l and risk movements in the format of a title and granular details which explains the title.
The title format is : Driven by <p&l amount> <financial factor> <P/L> to PL on <currency> Curve on Portfolio <portfolio name> 
The granular details format is : <p&l amount> <currency> due to <market_change> <market_change_direction> <financial factor> in the <tenor> node at the <long/short> <risk position>.""",
                                    max_retries=5,
                                    wait_time=30,
                                    deployed_gcp=False
                                    )

gemini_all_tenor_summary_engine_simple = GeminiSimpleChatEngine(
                                    model_name="gemini-2.0-flash-001",
                                    temperature=0.5,
                                    max_output_tokens=256,
                                    systemInstructions="""You are financial expert. You will be given some tenor wise profit and loss movements. 
Your task is to summarize the each tenor details in a concise manner.""",
                                    max_retries=5,
                                    wait_time=30,
                                    deployed_gcp=False
                                    )

2025-03-20 21:38:36,329 - DEBUG ==> Initialized GeminiModel with model gemini-2.0-flash-001 , project openserve-0, location us-central1
2025-03-20 21:38:36,554 - DEBUG ==> Initialized GeminiModel with model gemini-2.0-flash-001 , project openserve-0, location us-central1


In [ ]:
def tenor_wise_summary_agent(commentaries: str):
    res =  tenor_wise_summary_engnine.run([
        f"User provided the following commentaries: {commentaries}",
        f"Now please provide a tenor wise summary of the profit and loss movements.",
        f"The summary should be extensive and should be understandable by a financial expert.",
        f"Don't just copy the commentaries, do some comparison and analysis both across and within tenors. If there are no entries in a tenor, please mention that in the summary.",
        f"You must use the tool 'TenorWiseFinSummary' to provide the summary following the arguments structure."
        ]
    )

    all_tenor_res = "\n".join([f"{k}: {v}" for k, v in res[0].items()])
    
    final_summary = gemini_all_tenor_summary_engine_simple.run([
        f"These are the tenor wise summaries: {all_tenor_res}",
        f"Now please provide a concise summary of the profit and loss movements in each tenor.",
        "The summary should be concise and should be understandable by a financial expert.",
        "Don't just copy the tenor wise summaries, do some comparison and analysis both across and within tenors. Dont use any bullet points or list format. Write in a paragraph format.",
        ]
    )

    return final_summary

def summarize_financial_commentaries(commentaries: List[Any]):
    ans = ""
    for c in commentaries:
        ans+= c["header"] + "\n"
        ans += tenor_wise_summary_agent(c)+ "\n\n"
    ans = ans.strip()
    return ans

In [70]:
sample_financial_commentaries = [
    {
        "header": "Driven by 0.57m IRDelta Loss to PL on USD Curve on Portfolio Florida Rates Macro",
        "footer":"""1770001 EUR due to 8.6 widening in the 10y node at the long 20251 EUR risk position.\n
            1821213 EUR due to 8.6 widening in the 1y node at the long 20211 EUR risk position.\n
            919231 EUR due to 1.2 tightening in the 2y node at the long 4251 EUR risk position"""
    },
    {
        "header": "Driven by -0.98m IRDelta Loss to PL on USD Curve on Portfolio Hedging Rates Macro",
        "footer": """199901 EUR due to 1.2 tightening in the 2y node at the short 4251 EUR risk position.\n
            123123 EUR due to 21.2 widening in the 1y node at the short 20211 EUR risk position.\n
            1123123 EUR due to 1.2 tightening in the 2y node at the short 4251 EUR risk position."""
    }
]

In [71]:
res = summarize_financial_commentaries(sample_financial_commentaries)

In [72]:
print(res)

Driven by 0.57m IRDelta Loss to PL on USD Curve on Portfolio Florida Rates Macro
The portfolio experienced losses across all tenors. The short term (tenor < 2y) suffered the most significant loss of 1.82m EUR, driven by an 8.6 widening in the 1y node impacting a long risk position. This single factor was the primary driver of overall portfolio P&L in the short term. The mid-term (2y < tenor < 10y) incurred a loss of 0.92m EUR, stemming from a 1.2 tightening in the 2y node, also impacting a long risk position. Interestingly, while both short and long term losses were driven by widening spreads, the mid-term loss was caused by tightening. The long term (tenor > 10y) experienced a smaller loss of 0.18m EUR, attributed to an 8.6 widening in the 10y node affecting a long risk position. The consistent long risk positioning across all tenors suggests a potential vulnerability to adverse movements in interest rate spreads, particularly widenings in the short and long end of the curve.


Driven